<a href="https://colab.research.google.com/github/omarfathyz8/Teeth-Classification/blob/main/TeethClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teeth Classification

##  Imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
import shutil

## Loading Datasets

#### Drive Mounting

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#### Unzip Dataset Folder

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/CellulaTechnologies/ComputerVision/TeethDataset.zip"
EXTRACT_DIR = "/content/TeethDataset"

In [ ]:
if not os.path.exists(EXTRACT_DIR):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)

In [ ]:
print("Extracted to:", EXTRACT_DIR)

Extracted to: /content/TeethDataset


#### Data Directories

In [ ]:
DATA_DIR = "/content/TeethDataset/Teeth_Dataset"
TRAIN_DIR = os.path.join(DATA_DIR, "Training")
TEST_DIR = os.path.join(DATA_DIR, "Testing")
VAL_DIR = os.path.join(DATA_DIR, 'Validation')

In [ ]:
print('Training classes   :', os.listdir(TRAIN_DIR))
print('Testing classes    :', os.listdir(TEST_DIR))
print('Validation classes :', os.listdir(VAL_DIR))

Training classes   : ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']
Testing classes    : ['output_image.jpg', 'outputs', 'MC', 'OC', 'OLP', 'Gum', 'output', 'OT', 'CaS', 'CoS', 'out']
Validation classes : ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']


#### Remove Extra Classes from Test Folder

In [ ]:
# Allowed class names
allowed_classes = ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']

# Loop through test_dir and remove anything not in allowed_classes
for item in os.listdir(TEST_DIR):
    item_path = os.path.join(TEST_DIR, item)
    if item not in allowed_classes:
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)  # remove folder
        else:
            os.remove(item_path)      # remove file
        print(f"Removed: {item}")

print("\nCleaned test directory successfully.\nClasses now:", os.listdir(TEST_DIR))

Removed: output_image.jpg
Removed: outputs
Removed: output
Removed: out

Cleaned test directory successfully.
Classes now: ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']
